In [12]:
from pymongo import MongoClient
import numpy as np

In [19]:
def getLangRatio(dbName, collectionName):
    
    collection = MongoClient("localhost", 27017)[dbName][collectionName]
    cursor = collection.find()
    user_langs = {}
    for tweet in cursor:
        lang = 1 if tweet["lang"] == "en" else 0
        user_id = tweet["user"]["id"]
        if user_id in user_langs:
            user_langs[user_id].append(lang)
        else:
            user_langs[user_id] = [lang]
    return user_langs

In [28]:
user_langs = getLangRatio("patients", "BPD_clean")

In [33]:
BPD_lang_ratios = [(user_id, np.sum(langs_ratio)/len(langs_ratio)) for user_id, langs_ratio in user_langs.items()]


In [37]:
BPD_lang_ratios = sorted(BPD_lang_ratios, key=lambda x: x[1])

for user_id, en_ratio in BPD_lang_ratios:
    if en_ratio < 0.75:
        print(user_id, en_ratio)

3262241903 0.0
944577823 0.119863013699
269690229 0.220713609094
2413555298 0.31308411215
3324571407 0.32
3263225837 0.333333333333
3077233757 0.380952380952
169951404 0.415136755339
244021118 0.430107526882
45229455 0.495652173913
3194839751 0.5
116439735 0.510220125786
510585906 0.550855085509
3227127058 0.551724137931
374595521 0.571428571429
2954272784 0.57215007215
2292426456 0.6
558473669 0.621212121212
202200525 0.644128113879
2936388081 0.654624277457
49450953 0.700718525461
2251489214 0.714285714286
3228645394 0.717948717949
3297759413 0.728155339806
58007127 0.731600375822
287681418 0.747753973739


In [38]:
user_langs = getLangRatio("patients", "bipolar_clean")
bipolar_lang_ratios = [(user_id, np.sum(langs_ratio)/len(langs_ratio)) for user_id, langs_ratio in user_langs.items()]
bipolar_lang_ratios = sorted(bipolar_lang_ratios, key=lambda x: x[1])

for user_id, en_ratio in bipolar_lang_ratios:
    if en_ratio < 0.75:
        print(user_id, en_ratio)

933392432 0.0
153294609 0.0172705722994
3029977273 0.0310130944176
2827792882 0.0769230769231
809571888 0.0880503144654
2499362502 0.127009646302
3367338663 0.173913043478
1229003286 0.185840707965
2286202992 0.188405797101
2984096968 0.2
3120472413 0.210365853659
504686644 0.261935483871
2921022655 0.317073170732
368501489 0.354838709677
553466344 0.385796545106
192116423 0.39165085389
3182719408 0.4
1962364130 0.407407407407
2425691041 0.425634824667
2600004696 0.439393939394
3041204445 0.475409836066
70032674 0.480043149946
3238987287 0.5
3076152329 0.539748953975
1015217155 0.558333333333
962084311 0.561849710983
1247445650 0.591133004926
1408777951 0.593103448276
206804977 0.608804402201
88995343 0.612752721617
29587967 0.61733615222
348873140 0.617647058824
582230868 0.624203821656
101816365 0.625
2751352106 0.636363636364
2950280698 0.645161290323
770070936 0.646706586826
102816119 0.655367231638
28725721 0.661290322581
3307369534 0.666666666667
2531462476 0.666666666667
3322592

In [ ]:
lang_ratios